In [37]:
import itertools
import httpx
import pandas as pd

API_URL = "https://saisonmanager.de/api/v2"

leagues_url = API_URL + "/leagues.json"
league_url = API_URL + "/leagues/{league_id}.json"
standings_url = API_URL + "/leagues/{league_id}/table.json"
scorers_url = API_URL + "/leagues/{league_id}/scorer.json"
schedule_url = API_URL + "/leagues/{league_id}/schedule.json"
game_url = API_URL + "/games/{game_id}.json"


In [19]:
leagues = pd.read_json(leagues_url)

# filter competitions by name (hopefully doesn't filter out something by accident)
leagues = leagues[
    (leagues.game_operation == "Floorball Deutschland")
    & (~leagues.name.str.contains("damen", case=False))
    & (~leagues.name.str.contains("junior", case=False))
    & (~leagues.name.str.contains("kleinfeld", case=False))
    & (~leagues.name.str.contains("KF"))
]

leagues.sort_values(by="season", ascending=False)

,id,operation_id,game_operation,season,name,order_key,link,link_schedule
753,1446,1,Floorball Deutschland,15,2. FBL Herren Ost,50,/leagues/1446.json,/leagues/1446/schedule.json
751,1444,1,Floorball Deutschland,15,2. FBL Herren Nord/West,30,/leagues/1444.json,/leagues/1444/schedule.json
747,1513,1,Floorball Deutschland,15,1. FBL Herren - Playoffs,11,/leagues/1513.json,/leagues/1513/schedule.json
748,1514,1,Floorball Deutschland,15,1. FBL Herren - Playdowns,12,/leagues/1514.json,/leagues/1514/schedule.json
746,1442,1,Floorball Deutschland,15,1. FBL Herren,10,/leagues/1442.json,/leagues/1442/schedule.json
...,...,...,...,...,...,...,...,...
896,118,1,Floorball Deutschland,6,Relegation 1./2. FBL Herren,16,/leagues/118.json,/leagues/118/schedule.json
895,285,1,Floorball Deutschland,6,Play-Offs 2. FBL Herren,15,/leagues/285.json,/leagues/285/schedule.json
894,33,1,Floorball Deutschland,6,Play-Down 1. FBL Herren,14,/leagues/33.json,/leagues/33/schedule.json
892,319,1,Floorball Deutschland,6,Play-Offs 1. FBL Herren,11,/leagues/319.json,/leagues/319/schedule.json


In [36]:
# collect schedule data for the leagues; need to check that the URL can
# be reached beforehand because a fewfail with a 500 Internal Server Error
schedules = {
    league_id: pd.read_json(schedule_url.format(league_id=league_id))
    for league_id in leagues.id
    if httpx.head(schedule_url.format(league_id=league_id)).is_success
}

In [104]:
# create additional column containing league ids
league_ids = pd.Series(
    itertools.chain(
        *[
            itertools.repeat(league_id, len(schedule))
            for league_id, schedule in schedules.items()
        ]
    ),
    name="league_id",
)

# create matches dataframe from the combined schedules
matches = pd.concat(schedules.values(), ignore_index=True)

# unpack the JSON strings in the result column into a dataframe
results = pd.json_normalize(matches["result"])


# use nullable integer type
results["home_goals"] = results["home_goals"].astype("Int64")
results["guest_goals"] = results["guest_goals"].astype("Int64")

# remove JSON result column
matches = matches.drop(columns=["result"])

# combine the three dataframes into one with all the columns
matches = pd.concat([league_ids, results, matches.reset_index()], axis="columns")
matches

,league_id,home_goals,guest_goals,home_goals_period,guest_goals_period,forfait,overtime,postfix.short,postfix.long,index,...,group_identifier,series_title,series_number,home_team_filling_rule,home_team_filling_title,home_team_filling_parameter,guest_team_filling_rule,guest_team_filling_title,guest_team_filling_parameter,result_string
0,601,7,15,"[3, 3, 1, 0]","[3, 1, 11, 0]",False,False,,,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7:15
1,601,6,15,"[3, 2, 1, 0]","[1, 7, 7, 0]",False,False,,,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6:15
2,601,0,0,"[1, 0, -1, 0]","[3, 4, -7, 0]",False,False,,,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0:0
3,601,5,4,"[1, 2, 1, 1]","[0, 3, 1, 0]",False,True,n.V.,nach Verlängerung,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5:4 n.V.
4,601,13,6,"[0, 4, 9, 0]","[1, 3, 2, 0]",False,False,,,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13:6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3113,516,16,2,"[3, 6, 7, 0]","[1, 1, 0, 0]",False,False,,,3113,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16:2
3114,516,7,5,"[3, 1, 3, 0]","[1, 2, 2, 0]",False,False,,,3114,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7:5
3115,516,2,3,"[0, 0, 2, 0]","[0, 0, 2, 1]",False,True,n.V.,nach Verlängerung,3115,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2:3 n.V.
3116,516,7,10,"[0, 2, 5, 0]","[4, 3, 3, 0]",False,False,,,3116,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7:10
